In [ ]:
!pip install tensorflow
!pip install scikit-learn

In [ ]:
# Import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report

In [ ]:
# Function to build the CNN model
def build_model(width, height, depth, classes):
    # Initialize the model
    model = Sequential()
    inputShape = (height, width, depth)

    # First set of CONV => RELU => POOL layers
    model.add(Conv2D(32, (5, 5), padding="same", input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Second set of CONV => RELU => POOL layers
    model.add(Conv2D(32, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # First set of FC => RELU layers
    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation("relu"))
    model.add(Dropout(0.5))

    # Second set of FC => RELU layers
    model.add(Dense(64))
    model.add(Activation("relu"))
    model.add(Dropout(0.5))

    # Softmax classifier
    model.add(Dense(classes))
    model.add(Activation("softmax"))

    # Return the constructed network architecture
    return model

In [ ]:
# Initialize the initial learning rate, number of epochs to train for, and batch size
INIT_LR = 1e-3
EPOCHS = 10
BS = 128

In [ ]:
# Grab the MNIST dataset
print("[INFO] accessing MNIST...")
((trainData, trainLabels), (testData, testLabels)) = mnist.load_data()

[INFO] accessing MNIST...
11490434/11490434 [==============================] - 2s 0us/step


In [ ]:
# Add a channel (i.e., grayscale) dimension to the digits
trainData = trainData.reshape((trainData.shape[0], 28, 28, 1))
testData = testData.reshape((testData.shape[0], 28, 28, 1))

In [ ]:
# Scale data to the range of [0, 1]
trainData = trainData.astype("float32") / 255.0
testData = testData.astype("float32") / 255.0

In [ ]:
# Convert the labels from integers to vectors
le = LabelBinarizer()
trainLabels = le.fit_transform(trainLabels)
testLabels = le.transform(testLabels)

In [ ]:
# Initialize the optimizer and model
print("[INFO] compiling model...")
opt = Adam(learning_rate=INIT_LR)
model = build_model(width=28, height=28, depth=1, classes=10)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

[INFO] compiling model...


In [ ]:
# Train the network
print("[INFO] training network...")
H = model.fit(
    trainData, trainLabels,
    validation_data=(testData, testLabels),
    batch_size=BS,
    epochs=EPOCHS,
    verbose=1
)

[INFO] training network...
Epoch 1/10
469/469 [==============================] - 11s 9ms/step - loss: 0.6853 - accuracy: 0.7770 - val_loss: 0.0963 - val_accuracy: 0.9743
Epoch 2/10
469/469 [==============================] - 5s 11ms/step - loss: 0.2547 - accuracy: 0.9246 - val_loss: 0.0662 - val_accuracy: 0.9821
Epoch 3/10
469/469 [==============================] - 5s 11ms/step - loss: 0.2009 - accuracy: 0.9420 - val_loss: 0.0462 - val_accuracy: 0.9863
Epoch 4/10
469/469 [==============================] - 3s 5ms/step - loss: 0.1715 - accuracy: 0.9507 - val_loss: 0.0463 - val_accuracy: 0.9867
Epoch 5/10
469/469 [==============================] - 2s 5ms/step - loss: 0.1505 - accuracy: 0.9569 - val_loss: 0.0429 - val_accuracy: 0.9874
Epoch 6/10
469/469 [==============================] - 2s 5ms/step - loss: 0.1337 - accuracy: 0.9619 - val_loss: 0.0394 - val_accuracy: 0.9876
Epoch 7/10
469/469 [==============================] - 3s 6ms/step - loss: 0.1319 - accuracy: 0.9621 - val_loss: 0.0390

In [ ]:
# Evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testData)
print(classification_report(
    testLabels.argmax(axis=1),
    predictions.argmax(axis=1),
    target_names=[str(x) for x in le.classes_]
))

[INFO] evaluating network...
313/313 [==============================] - 1s 3ms/step
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       980
           1       0.99      1.00      0.99      1135
           2       0.99      1.00      0.99      1032
           3       0.99      0.99      0.99      1010
           4       0.99      0.99      0.99       982
           5       0.99      0.99      0.99       892
           6       0.99      0.99      0.99       958
           7       0.99      0.99      0.99      1028
           8       0.99      0.99      0.99       974
           9       0.99      0.98      0.99      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000



In [ ]:
# Serialize the model to disk
print("[INFO] serializing digit model...")
model.save("ocr.h5", save_format="h5")

[INFO] serializing digit model...


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
